In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.8))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [7]:
suffix = 'sim08.csv'
n_samples = 500

### BERT-based-uncased

In [8]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8114446529080676
RF 0.7335834896810507
KNN 0.6801125703564728
SVC 0.8030018761726079


In [10]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [11]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [12]:
runAttack(bert_model_wrapper_lr_mr, n_samples, 'attack_results_new/bertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                         | 5/500 [01:16<2:06:44, 15.36s/it]textattack: Saving checkpoint under "checkpoints\1720231672053.ta.chkpt" at 2024-07-06 10:07:52 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 4 / 4 / 10:   2%|▍                       | 10/500 [01:25<1:09:47,  8.55s/it]textattack: Saving checkpoint under "checkpoints\1720231680689.ta.chkpt" at 2024-07-06 10:08:00 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 7 / 4 / 15:   3%|▊                         | 15/500 [01:41<54:27,  6.74s/it]textattack: Saving checkpoint under "checkpoints\1720231696296.ta.chkpt" at 2024-07-06 10:08:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 269 / 118 / 98 / 485:  97%|██████████████████▍| 485/500 [13:47<00:25,  1.71s/it]textattack: Saving checkpoint under "checkpoints\1720232422693.ta.chkpt" at 2024-07-06 10:20:22 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 273 / 118 / 99 / 490:  98%|██████████████████▌| 490/500 [13:55<00:17,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1720232430512.ta.chkpt" at 2024-07-06 10:20:30 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 277 / 119 / 99 / 495:  99%|██████████████████▊| 495/500 [14:02<00:08,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1720232437488.ta.chkpt" at 2024-07-06 10:20:37 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 281 / 120 / 99 / 500: 100%|███████████████████| 500/500 [14:11<00:00,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1720232446839.ta.chkpt" at 2024-07-06 10:20:46 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 281 / 120 / 99 / 500: 100%|███████████████████| 500/500 [14:11<00:00,  1.70s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 281    |
| Number of failed attacks:     | 120    |
| Number of skipped attacks:    | 99     |
| Original accuracy:            | 80.2%  |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 70.07% |
| Average perturbed word %:     | 20.07% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 35.21  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_rf_mr, n_samples, 'attack_results_new/bertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 0 / 3 / 2 / 5:   1%|▎                           | 5/500 [00:23<38:13,  4.63s/it]textattack: Saving checkpoint under "checkpoints\1720232471160.ta.chkpt" at 2024-07-06 10:21:11 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 3 / 5 / 10:   2%|▌                         | 10/500 [00:29<23:44,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1720232477057.ta.chkpt" at 2024-07-06 10:21:17 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 5 / 6 / 15:   3%|▊                         | 15/500 [00:40<21:48,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1720232488466.ta.chkpt" at 2024-07-06 10:21:28 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 251 / 101 / 133 / 485:  97%|█████████████████▍| 485/500 [11:01<00:20,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720233109645.ta.chkpt" at 2024-07-06 10:31:49 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 255 / 101 / 134 / 490:  98%|█████████████████▋| 490/500 [11:09<00:13,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720233117005.ta.chkpt" at 2024-07-06 10:31:57 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 255 / 101 / 135 / 491:  98%|█████████████████▋| 491/500 [11:09<00:12,  1.36s/it]

[Succeeded / Failed / Skipped / Total] 257 / 102 / 136 / 495:  99%|█████████████████▊| 495/500 [11:16<00:06,  1.37s/it]textattack: Saving checkpoint under "checkpoints\1720233124397.ta.chkpt" at 2024-07-06 10:32:04 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 260 / 103 / 137 / 500: 100%|██████████████████| 500/500 [11:21<00:00,  1.36s/it]textattack: Saving checkpoint under "checkpoints\1720233129819.ta.chkpt" at 2024-07-06 10:32:09 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 260 / 103 / 137 / 500: 100%|██████████████████| 500/500 [11:21<00:00,  1.36s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 260    |
| Number of failed attacks:     | 103    |
| Number of skipped attacks:    | 137    |
| Original accuracy:            | 72.6%  |
| Accuracy under attack:        | 20.6%  |
| Attack success rate:          | 71.63% |
| Average perturbed word %:     | 15.8%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 32.95  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_knn_mr, n_samples, 'attack_results_new/bertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:10<17:40,  2.14s/it]textattack: Saving checkpoint under "checkpoints\1720233140642.ta.chkpt" at 2024-07-06 10:32:20 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 6/500 [00:10<14:57,  1.82s/it]

[Succeeded / Failed / Skipped / Total] 5 / 1 / 4 / 10:   2%|▌                         | 10/500 [00:16<13:27,  1.65s/it]textattack: Saving checkpoint under "checkpoints\1720233146409.ta.chkpt" at 2024-07-06 10:32:26 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 2 / 6 / 15:   3%|▊                         | 15/500 [00:25<13:38,  1.69s/it]textattack: Saving checkpoint under "checkpoints\1720233155237.ta.chkpt" at 2024-07-06 10:32:35 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 304 / 34 / 147 / 485:  97%|██████████████████▍| 485/500 [07:35<00:14,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720233585909.ta.chkpt" at 2024-07-06 10:39:45 after 485 attacks.
[Succeeded / Failed / Skipped / Total] 304 / 34 / 148 / 486:  97%|██████████████████▍| 486/500 [07:36<00:13,  1.07it/s]

[Succeeded / Failed / Skipped / Total] 305 / 36 / 149 / 490:  98%|██████████████████▌| 490/500 [07:43<00:09,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720233593441.ta.chkpt" at 2024-07-06 10:39:53 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 305 / 36 / 150 / 491:  98%|██████████████████▋| 491/500 [07:43<00:08,  1.06it/s]

[Succeeded / Failed / Skipped / Total] 307 / 37 / 151 / 495:  99%|██████████████████▊| 495/500 [07:47<00:04,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720233597223.ta.chkpt" at 2024-07-06 10:39:57 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 309 / 38 / 153 / 500: 100%|███████████████████| 500/500 [07:51<00:00,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720233601723.ta.chkpt" at 2024-07-06 10:40:01 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 309 / 38 / 153 / 500: 100%|███████████████████| 500/500 [07:51<00:00,  1.06it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 309    |
| Number of failed attacks:     | 38     |
| Number of skipped attacks:    | 153    |
| Original accuracy:            | 69.4%  |
| Accuracy under attack:        | 7.6%   |
| Attack success rate:          | 89.05% |
| Average perturbed word %:     | 9.91%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 24.98  |
+-------------------------------+--------+


In [15]:
runAttack(bert_model_wrapper_svc_mr, n_samples, 'attack_results_new/bertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvcsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                           | 5/500 [00:15<26:14,  3.18s/it]textattack: Saving checkpoint under "checkpoints\1720233617751.ta.chkpt" at 2024-07-06 10:40:17 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 3 / 7:   1%|▍                           | 7/500 [00:16<18:51,  2.29s/it]

[Succeeded / Failed / Skipped / Total] 2 / 4 / 4 / 10:   2%|▌                         | 10/500 [00:21<17:43,  2.17s/it]textattack: Saving checkpoint under "checkpoints\1720233623548.ta.chkpt" at 2024-07-06 10:40:23 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 7 / 4 / 15:   3%|▊                         | 15/500 [00:37<20:23,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1720233639690.ta.chkpt" at 2024-07-06 10:40:39 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 274 / 115 / 96 / 485:  97%|██████████████████▍| 485/500 [12:17<00:22,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720234339548.ta.chkpt" at 2024-07-06 10:52:19 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 278 / 115 / 97 / 490:  98%|██████████████████▌| 490/500 [12:24<00:15,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720234346642.ta.chkpt" at 2024-07-06 10:52:26 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 282 / 116 / 97 / 495:  99%|██████████████████▊| 495/500 [12:31<00:07,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720234353174.ta.chkpt" at 2024-07-06 10:52:33 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 286 / 117 / 97 / 500: 100%|███████████████████| 500/500 [12:40<00:00,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720234362315.ta.chkpt" at 2024-07-06 10:52:42 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 286 / 117 / 97 / 500: 100%|███████████████████| 500/500 [12:40<00:00,  1.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 286    |
| Number of failed attacks:     | 117    |
| Number of skipped attacks:    | 97     |
| Original accuracy:            | 80.6%  |
| Accuracy under attack:        | 23.4%  |
| Attack success rate:          | 70.97% |
| Average perturbed word %:     | 19.33% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 34.5   |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [16]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [18]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [19]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [20]:
runAttack(disbert_model_wrapper_lr_mr, n_samples, 'attack_results_new/disbertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 2 / 1 / 5:   1%|▎                           | 5/500 [00:11<18:24,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1720234527873.ta.chkpt" at 2024-07-06 10:55:27 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 2 / 3 / 7:   1%|▍                           | 7/500 [00:11<13:19,  1.62s/it]

[Succeeded / Failed / Skipped / Total] 5 / 2 / 3 / 10:   2%|▌                         | 10/500 [00:15<12:24,  1.52s/it]textattack: Saving checkpoint under "checkpoints\1720234531905.ta.chkpt" at 2024-07-06 10:55:31 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 5 / 4 / 15:   3%|▊                         | 15/500 [00:23<12:35,  1.56s/it]textattack: Saving checkpoint under "checkpoints\1720234540093.ta.chkpt" at 2024-07-06 10:55:40 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 275 / 108 / 102 / 485:  97%|█████████████████▍| 485/500 [07:17<00:13,  1.11it/s]textattack: Saving checkpoint under "checkpoints\1720234954291.ta.chkpt" at 2024-07-06 11:02:34 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 278 / 109 / 103 / 490:  98%|█████████████████▋| 490/500 [07:20<00:08,  1.11it/s]textattack: Saving checkpoint under "checkpoints\1720234957463.ta.chkpt" at 2024-07-06 11:02:37 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 278 / 109 / 104 / 491:  98%|█████████████████▋| 491/500 [07:20<00:08,  1.11it/s]

[Succeeded / Failed / Skipped / Total] 281 / 110 / 104 / 495:  99%|█████████████████▊| 495/500 [07:26<00:04,  1.11it/s]textattack: Saving checkpoint under "checkpoints\1720234963338.ta.chkpt" at 2024-07-06 11:02:43 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 286 / 110 / 104 / 500: 100%|██████████████████| 500/500 [07:33<00:00,  1.10it/s]textattack: Saving checkpoint under "checkpoints\1720234970386.ta.chkpt" at 2024-07-06 11:02:50 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 286 / 110 / 104 / 500: 100%|██████████████████| 500/500 [07:33<00:00,  1.10it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 286    |
| Number of failed attacks:     | 110    |
| Number of skipped attacks:    | 104    |
| Original accuracy:            | 79.2%  |
| Accuracy under attack:        | 22.0%  |
| Attack success rate:          | 72.22% |
| Average perturbed word %:     | 17.92% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 33.42  |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_rf_mr, n_samples, 'attack_results_new/disbertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 1 / 3 / 1 / 5:   1%|▎                           | 5/500 [00:12<20:08,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1720234982668.ta.chkpt" at 2024-07-06 11:03:02 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 3 / 2 / 6:   1%|▎                           | 6/500 [00:12<16:57,  2.06s/it]

[Succeeded / Failed / Skipped / Total] 3 / 5 / 2 / 10:   2%|▌                         | 10/500 [00:18<15:25,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1720234989357.ta.chkpt" at 2024-07-06 11:03:09 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 8 / 3 / 15:   3%|▊                         | 15/500 [00:27<14:41,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720234997716.ta.chkpt" at 2024-07-06 11:03:17 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 245 / 123 / 117 / 485:  97%|█████████████████▍| 485/500 [07:39<00:14,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720235429834.ta.chkpt" at 2024-07-06 11:10:29 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 248 / 124 / 118 / 490:  98%|█████████████████▋| 490/500 [07:44<00:09,  1.05it/s]textattack: Saving checkpoint under "checkpoints\1720235435303.ta.chkpt" at 2024-07-06 11:10:35 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 248 / 124 / 119 / 491:  98%|█████████████████▋| 491/500 [07:44<00:08,  1.06it/s]

[Succeeded / Failed / Skipped / Total] 250 / 125 / 120 / 495:  99%|█████████████████▊| 495/500 [07:47<00:04,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720235438366.ta.chkpt" at 2024-07-06 11:10:38 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 253 / 127 / 120 / 500: 100%|██████████████████| 500/500 [07:55<00:00,  1.05it/s]textattack: Saving checkpoint under "checkpoints\1720235445882.ta.chkpt" at 2024-07-06 11:10:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 253 / 127 / 120 / 500: 100%|██████████████████| 500/500 [07:55<00:00,  1.05it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 253    |
| Number of failed attacks:     | 127    |
| Number of skipped attacks:    | 120    |
| Original accuracy:            | 76.0%  |
| Accuracy under attack:        | 25.4%  |
| Attack success rate:          | 66.58% |
| Average perturbed word %:     | 17.82% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 35.17  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_knn_mr, n_samples, 'attack_results_new/disbertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:   1%|▎                           | 5/500 [00:10<18:07,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720235456979.ta.chkpt" at 2024-07-06 11:10:56 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 2 / 2 / 7:   1%|▍                           | 7/500 [00:11<13:03,  1.59s/it]

[Succeeded / Failed / Skipped / Total] 4 / 4 / 2 / 10:   2%|▌                         | 10/500 [00:16<13:45,  1.68s/it]textattack: Saving checkpoint under "checkpoints\1720235462840.ta.chkpt" at 2024-07-06 11:11:02 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 6 / 3 / 15:   3%|▊                         | 15/500 [00:24<13:09,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1720235470407.ta.chkpt" at 2024-07-06 11:11:10 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 253 / 104 / 128 / 485:  97%|█████████████████▍| 485/500 [06:55<00:12,  1.17it/s]textattack: Saving checkpoint under "checkpoints\1720235861473.ta.chkpt" at 2024-07-06 11:17:41 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 256 / 105 / 129 / 490:  98%|█████████████████▋| 490/500 [06:59<00:08,  1.17it/s]textattack: Saving checkpoint under "checkpoints\1720235865304.ta.chkpt" at 2024-07-06 11:17:45 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 257 / 107 / 131 / 495:  99%|█████████████████▊| 495/500 [07:05<00:04,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720235871006.ta.chkpt" at 2024-07-06 11:17:51 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 257 / 107 / 132 / 496:  99%|█████████████████▊| 496/500 [07:05<00:03,  1.17it/s]

[Succeeded / Failed / Skipped / Total] 258 / 110 / 132 / 500: 100%|██████████████████| 500/500 [07:11<00:00,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720235877378.ta.chkpt" at 2024-07-06 11:17:57 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 258 / 110 / 132 / 500: 100%|██████████████████| 500/500 [07:11<00:00,  1.16it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 258    |
| Number of failed attacks:     | 110    |
| Number of skipped attacks:    | 132    |
| Original accuracy:            | 73.6%  |
| Accuracy under attack:        | 22.0%  |
| Attack success rate:          | 70.11% |
| Average perturbed word %:     | 10.28% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 31.24  |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_svc_mr, n_samples, 'attack_results_new/disbertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvcsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   1%|▎                           | 5/500 [00:11<19:27,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1720235889291.ta.chkpt" at 2024-07-06 11:18:09 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 3 / 7:   1%|▍                           | 7/500 [00:11<14:01,  1.71s/it]

[Succeeded / Failed / Skipped / Total] 6 / 1 / 3 / 10:   2%|▌                         | 10/500 [00:15<13:00,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720235893420.ta.chkpt" at 2024-07-06 11:18:13 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 5 / 4 / 15:   3%|▊                         | 15/500 [00:24<13:10,  1.63s/it]textattack: Saving checkpoint under "checkpoints\1720235901945.ta.chkpt" at 2024-07-06 11:18:21 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 283 / 98 / 104 / 485:  97%|██████████████████▍| 485/500 [07:45<00:14,  1.04it/s]textattack: Saving checkpoint under "checkpoints\1720236343402.ta.chkpt" at 2024-07-06 11:25:43 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 285 / 99 / 106 / 490:  98%|██████████████████▌| 490/500 [07:49<00:09,  1.04it/s]textattack: Saving checkpoint under "checkpoints\1720236346918.ta.chkpt" at 2024-07-06 11:25:46 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 285 / 99 / 107 / 491:  98%|██████████████████▋| 491/500 [07:49<00:08,  1.05it/s]

[Succeeded / Failed / Skipped / Total] 288 / 100 / 107 / 495:  99%|█████████████████▊| 495/500 [07:55<00:04,  1.04it/s]textattack: Saving checkpoint under "checkpoints\1720236352505.ta.chkpt" at 2024-07-06 11:25:52 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 293 / 100 / 107 / 500: 100%|██████████████████| 500/500 [08:01<00:00,  1.04it/s]textattack: Saving checkpoint under "checkpoints\1720236359274.ta.chkpt" at 2024-07-06 11:25:59 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 293 / 100 / 107 / 500: 100%|██████████████████| 500/500 [08:01<00:00,  1.04it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 293    |
| Number of failed attacks:     | 100    |
| Number of skipped attacks:    | 107    |
| Original accuracy:            | 78.6%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 74.55% |
| Average perturbed word %:     | 18.33% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 33.26  |
+-------------------------------+--------+


### CLIP

In [24]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [26]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [27]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [28]:
runAttack(clip_model_wrapper_lr_mr, n_samples, 'attack_results_new/clipmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   1%|▎                           | 5/500 [00:12<20:32,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1720236792014.ta.chkpt" at 2024-07-06 11:33:12 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 1 / 6:   1%|▎                           | 6/500 [00:12<17:17,  2.10s/it]

[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10:   2%|▌                         | 10/500 [00:16<13:31,  1.66s/it]textattack: Saving checkpoint under "checkpoints\1720236796127.ta.chkpt" at 2024-07-06 11:33:16 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 3 / 1 / 15:   3%|▊                        | 15/500 [00:24<13:27,  1.67s/it]textattack: Saving checkpoint under "checkpoints\1720236804551.ta.chkpt" at 2024-07-06 11:33:24 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 291 / 67 / 127 / 485:  97%|██████████████████▍| 485/500 [06:46<00:12,  1.19it/s]textattack: Saving checkpoint under "checkpoints\1720237186501.ta.chkpt" at 2024-07-06 11:39:46 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 294 / 67 / 129 / 490:  98%|██████████████████▌| 490/500 [06:49<00:08,  1.20it/s]textattack: Saving checkpoint under "checkpoints\1720237189221.ta.chkpt" at 2024-07-06 11:39:49 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 294 / 67 / 130 / 491:  98%|██████████████████▋| 491/500 [06:49<00:07,  1.20it/s]

[Succeeded / Failed / Skipped / Total] 297 / 67 / 131 / 495:  99%|██████████████████▊| 495/500 [06:51<00:04,  1.20it/s]textattack: Saving checkpoint under "checkpoints\1720237191421.ta.chkpt" at 2024-07-06 11:39:51 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 302 / 67 / 131 / 500: 100%|███████████████████| 500/500 [06:56<00:00,  1.20it/s]textattack: Saving checkpoint under "checkpoints\1720237196524.ta.chkpt" at 2024-07-06 11:39:56 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 302 / 67 / 131 / 500: 100%|███████████████████| 500/500 [06:57<00:00,  1.20it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 302    |
| Number of failed attacks:     | 67     |
| Number of skipped attacks:    | 131    |
| Original accuracy:            | 73.8%  |
| Accuracy under attack:        | 13.4%  |
| Attack success rate:          | 81.84% |
| Average perturbed word %:     | 17.89% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 30.15  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_rf_mr, n_samples, 'attack_results_new/clipmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:13<22:04,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1720237210018.ta.chkpt" at 2024-07-06 11:40:10 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 3 / 2 / 7:   1%|▍                           | 7/500 [00:13<15:50,  1.93s/it]

[Succeeded / Failed / Skipped / Total] 5 / 3 / 2 / 10:   2%|▌                         | 10/500 [00:17<14:35,  1.79s/it]textattack: Saving checkpoint under "checkpoints\1720237214500.ta.chkpt" at 2024-07-06 11:40:14 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 5 / 3 / 15:   3%|▊                         | 15/500 [00:25<13:44,  1.70s/it]textattack: Saving checkpoint under "checkpoints\1720237222137.ta.chkpt" at 2024-07-06 11:40:22 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 246 / 95 / 144 / 485:  97%|██████████████████▍| 485/500 [07:40<00:14,  1.05it/s]textattack: Saving checkpoint under "checkpoints\1720237657541.ta.chkpt" at 2024-07-06 11:47:37 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 250 / 95 / 145 / 490:  98%|██████████████████▌| 490/500 [07:44<00:09,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720237660994.ta.chkpt" at 2024-07-06 11:47:40 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 250 / 95 / 146 / 491:  98%|██████████████████▋| 491/500 [07:44<00:08,  1.06it/s]

[Succeeded / Failed / Skipped / Total] 252 / 95 / 148 / 495:  99%|██████████████████▊| 495/500 [07:48<00:04,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720237665088.ta.chkpt" at 2024-07-06 11:47:45 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 252 / 95 / 149 / 496:  99%|██████████████████▊| 496/500 [07:48<00:03,  1.06it/s]

[Succeeded / Failed / Skipped / Total] 256 / 95 / 149 / 500: 100%|███████████████████| 500/500 [07:52<00:00,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720237669625.ta.chkpt" at 2024-07-06 11:47:49 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 256 / 95 / 149 / 500: 100%|███████████████████| 500/500 [07:53<00:00,  1.06it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 256    |
| Number of failed attacks:     | 95     |
| Number of skipped attacks:    | 149    |
| Original accuracy:            | 70.2%  |
| Accuracy under attack:        | 19.0%  |
| Attack success rate:          | 72.93% |
| Average perturbed word %:     | 16.03% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 32.59  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_knn_mr, n_samples, 'attack_results_new/clipmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:12<19:51,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1720237681774.ta.chkpt" at 2024-07-06 11:48:01 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 5 / 2 / 10:   2%|▌                         | 10/500 [00:18<15:05,  1.85s/it]textattack: Saving checkpoint under "checkpoints\1720237688209.ta.chkpt" at 2024-07-06 11:48:08 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 7 / 3 / 15:   3%|▊                         | 15/500 [00:27<14:39,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720237696931.ta.chkpt" at 2024-07-06 11:48:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 243 / 90 / 152 / 485:  97%|██████████████████▍| 485/500 [06:56<00:12,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720238086612.ta.chkpt" at 2024-07-06 11:54:46 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 247 / 90 / 153 / 490:  98%|██████████████████▌| 490/500 [07:01<00:08,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720238090786.ta.chkpt" at 2024-07-06 11:54:50 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 247 / 90 / 154 / 491:  98%|██████████████████▋| 491/500 [07:01<00:07,  1.17it/s]

[Succeeded / Failed / Skipped / Total] 248 / 91 / 156 / 495:  99%|██████████████████▊| 495/500 [07:02<00:04,  1.17it/s]textattack: Saving checkpoint under "checkpoints\1720238092452.ta.chkpt" at 2024-07-06 11:54:52 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 248 / 91 / 158 / 497:  99%|██████████████████▉| 497/500 [07:02<00:02,  1.18it/s]

[Succeeded / Failed / Skipped / Total] 250 / 92 / 158 / 500: 100%|███████████████████| 500/500 [07:06<00:00,  1.17it/s]textattack: Saving checkpoint under "checkpoints\1720238095934.ta.chkpt" at 2024-07-06 11:54:55 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 250 / 92 / 158 / 500: 100%|███████████████████| 500/500 [07:06<00:00,  1.17it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 250    |
| Number of failed attacks:     | 92     |
| Number of skipped attacks:    | 158    |
| Original accuracy:            | 68.4%  |
| Accuracy under attack:        | 18.4%  |
| Attack success rate:          | 73.1%  |
| Average perturbed word %:     | 10.61% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 30.94  |
+-------------------------------+--------+


In [31]:
runAttack(clip_model_wrapper_svc_mr, n_samples, 'attack_results_new/clipmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvcsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   1%|▎                           | 5/500 [00:12<20:30,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1720238108450.ta.chkpt" at 2024-07-06 11:55:08 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7:   1%|▍                           | 7/500 [00:12<14:45,  1.80s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   2%|▌                         | 10/500 [00:16<13:15,  1.62s/it]textattack: Saving checkpoint under "checkpoints\1720238112244.ta.chkpt" at 2024-07-06 11:55:12 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   3%|▊                        | 15/500 [00:22<12:12,  1.51s/it]textattack: Saving checkpoint under "checkpoints\1720238118684.ta.chkpt" at 2024-07-06 11:55:18 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 294 / 66 / 125 / 485:  97%|██████████████████▍| 485/500 [06:39<00:12,  1.21it/s]textattack: Saving checkpoint under "checkpoints\1720238495926.ta.chkpt" at 2024-07-06 12:01:35 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 296 / 66 / 128 / 490:  98%|██████████████████▌| 490/500 [06:42<00:08,  1.22it/s]textattack: Saving checkpoint under "checkpoints\1720238498737.ta.chkpt" at 2024-07-06 12:01:38 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 300 / 66 / 129 / 495:  99%|██████████████████▊| 495/500 [06:45<00:04,  1.22it/s]textattack: Saving checkpoint under "checkpoints\1720238501233.ta.chkpt" at 2024-07-06 12:01:41 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 305 / 66 / 129 / 500: 100%|███████████████████| 500/500 [06:50<00:00,  1.22it/s]textattack: Saving checkpoint under "checkpoints\1720238506751.ta.chkpt" at 2024-07-06 12:01:46 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 305 / 66 / 129 / 500: 100%|███████████████████| 500/500 [06:50<00:00,  1.22it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 305    |
| Number of failed attacks:     | 66     |
| Number of skipped attacks:    | 129    |
| Original accuracy:            | 74.2%  |
| Accuracy under attack:        | 13.2%  |
| Attack success rate:          | 82.21% |
| Average perturbed word %:     | 17.38% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 29.47  |
+-------------------------------+--------+
